In [4]:
from random import uniform, random
from math import e, sqrt, cos, pi
import numpy as np
import random as random
from scipy.optimize import minimize

In [8]:
class ABC():
    class Bee():
        
        def get_fitness(self, position, fx):
            cost = fx(position)
            if(cost >= 0):
                return 1/1+cost, cost
            return 1 + abs(cost), cost


        def __init__(self, position, fx) -> None:
            self.memory = position
            self.fitness, self.cost = self.get_fitness(self.memory,fx)
            self.fx = fx
        

        def find_neighbourhood(self, fx):
            neighbour = minimize(fx, self.memory, method='COBYLA').x
            fitness_neighbour, cost_neighbour = self.get_fitness(neighbour,fx)
            if(fitness_neighbour < self.fitness):
                self.memory = neighbour
                self.fitness = fitness_neighbour
                self.cost = cost_neighbour
                return 1
            else:
                return 0

        def set_memory(self, position):
            self.memory = position
            self.fitness, self.cost = self.get_fitness(position,self.fx)

        def get_memory(self):
            return self.memory

    class OnlookerBee(Bee):

        def __init__(self,position, fx) -> None:
            self.fx = fx
            super().__init__(position,fx)

        def comunicate(self, swarm):
            population_fitness = sum([bee.fitness for bee in swarm])
            bees_probabilities = [bee.fitness/population_fitness for bee in swarm]
            bee = np.random.choice(swarm, p=bees_probabilities)
            self.memory = bee.memory
            self.cost = bee.cost
            self.fitness = bee.fitness
            better_neighbourhood = super().find_neighbourhood(self.fx)
            if(better_neighbourhood == 1):
                if(self.cost < bee.cost):
                    bee.memory = self.memory
                    bee.cost = self.cost
                    bee.fitness = self.fitness


    class EmployedBee(Bee):

        def __init__(self,position, lower, upper, fx) -> None:
            super().__init__(position, fx)
            self.fx = fx
            self.lower = lower
            self.upper = upper
            self.tries = 0

        def move(self, limit):
            if(self.tries < limit):
                gain = super().find_neighbourhood( self.fx)
                if(gain == 0):
                    self.tries += 1
            else:
                self.tries = 0
                self.scout()

        def scout(self):
            super().set_memory(np.array([self.lower + random.uniform(0, 1)*(self.upper-self.lower) for _ in range(0, len(super().get_memory()))]))


    def __init__(self,  dimention, num_points, bonds, numlookers, fx) -> None:
        self.num_points = num_points
        self.fx = fx
        self.swarm = [self.EmployedBee(np.array([bonds[0] + random.uniform(0, 1)*(bonds[1]-bonds[0]) for _ in range(0, dimention)]), bonds[0], bonds[1], fx) for _ in range(0, num_points)]
        self.unlooker_bees = [self.OnlookerBee(np.ones(dimention, dtype = int), fx) for _ in range(0, numlookers)]
        
    def run(self, num_iterations, limit):
        def find_best():
            best_bee = self.swarm[0]
            best_cost = self.fx(best_bee.memory)
            for workerbee in self.swarm:
                cost_bee = workerbee.cost
                if(cost_bee < best_cost):
                    best_cost = cost_bee
                    best_bee = workerbee
            for unlookerbee in self.swarm:
                cost_bee = unlookerbee.cost
                if(cost_bee < best_cost):
                    best_cost = cost_bee
                    best_bee = unlookerbee
            return best_bee
        for unlookerbee in self.unlooker_bees:
            unlookerbee.comunicate(self.swarm)
        best_bee = find_best()
        for _ in range(0, num_iterations):
            for workerbee in self.swarm:
                workerbee.move( limit)
            for unlookerbee in self.unlooker_bees:
                unlookerbee.comunicate(self.swarm)
            act_best = find_best()
            if(act_best.cost < best_bee.cost):
                best_bee = act_best      
        return best_bee.memory

## TEST'S

### Himmelblau's function

![alt text](him.png "Title")

In [9]:
fx = lambda x : (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 -7)**2

In [10]:
abc =  ABC(dimention=2, num_points=30, bonds=[-5, 5], numlookers=30, fx=fx)
abc.run(num_iterations=200, limit=50)

array([2.99999711, 1.99998533])

### Ackley function

![alt text](akley.png "Title")

In [11]:
fx = lambda x : -20*e**(-0.2*sqrt(0.5*(x[0]**2 + x[1]**2)))-e**(0.5*(cos(2*pi*x[1])+cos(2*pi*x[0])))+e+20

In [12]:
abc = ABC(dimention=2, num_points=50, bonds=[-5, 5], numlookers=30, fx=fx)
abc.run(num_iterations=200, limit=20)

array([4.34083352e-05, 9.76766167e-06])

### Rastrigin

![alt text](ras.png "Title")

In [13]:
fx = lambda x : 10*2 + (x[0]**2 - 10*cos(2*pi*x[0])) + (x[1]**2 - 10*cos(2*pi*x[1]))

In [14]:
abc = ABC(dimention=2, num_points=50, bonds=[-5, 5], numlookers=30, fx=fx)
abc.run(num_iterations=200, limit=20)

array([1.37892074e-05, 3.93446149e-05])